## 1.Import data train_split as a pandas dataframe 

In [51]:
# insert train_split data as Pandas Dataframe Object and remove the df* variable

train_df = pd.read_csv(body)
train_df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,...,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,...,0.0,74.0,INDIVIDUAL,NaN,26th week,0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,...,0.0,NaN,INDIVIDUAL,NaN,9th week,0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,...,0.0,NaN,INDIVIDUAL,NaN,9th week,0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,...,0.0,NaN,INDIVIDUAL,NaN,135th week,0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,...,0.0,NaN,INDIVIDUAL,NaN,96th week,0,0.0,47159.0,47033.0,0


In [52]:
train_df.shape

(63999, 45)

## 2.Take the necessary pre processing steps

In [53]:
from sklearn import preprocessing
import numpy as np

def prep_data(WORKING_DF):
    encoded = pd.DataFrame()
    
    WORKING_DF.drop(columns=['batch_enrolled', 'desc', 'zip_code'], axis=1)
    # Mapping and encoding emp_length values
    scale_mapper = {np.nan:0, '< 1 year':1, '1 year':2, '2 years':3, '3 years':4, '4 years':5, '5 years':6, '6 years':7, '7 years':8, '8 years':9, '9 years':10, '10+ years':11}
    encoded['emp_length_encoded'] = WORKING_DF['emp_length'].replace(scale_mapper)

    # Encoding remaining ordinal variables
    grouped = WORKING_DF[['delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'pub_rec', 'total_acc', 'open_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'last_week_pay']]
    grouped = grouped.apply(preprocessing.LabelEncoder().fit_transform)
    encoded = pd.concat([encoded, grouped], axis=1)


    # One-hot encode nominal variables
    grouped = WORKING_DF[['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 'verification_status_joint', 'member_id']]
    grouped = pd.get_dummies(grouped)
    encoded = pd.concat([encoded, grouped], axis=1)

    # Append float columns to encoded df
    grouped = WORKING_DF[['annual_inc', 'dti', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
                          'collection_recovery_fee', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'open_acc', 'mths_since_last_major_derog']]

    # Fill NaN values with mean of each column
    fill_NaN = preprocessing.Imputer(missing_values=np.nan, strategy='mean', axis=0)
    imputed_df = pd.DataFrame(fill_NaN.fit_transform(grouped))
    imputed_df.columns = grouped.columns
    imputed_df.index = grouped.index

    encoded = pd.concat([encoded, imputed_df], axis=1)
    return encoded

In [54]:
encoded = prep_data(train_df)
encoded_test_df = prep_data(test_df)
encoded = encoded.drop(columns=['home_ownership_ANY'], axis=1)

In [55]:
encoded_test_df.head()

,emp_length_encoded,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,total_acc,open_acc,collections_12_mths_ex_med,acc_now_delinq,...,total_acc,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,open_acc,mths_since_last_major_derog
0,5,1,1,20,5329,0,51,16,0,0,...,53.0,3915.61,0.0,0.0,0.0,0.0,85230.0,45700.0,16.0,44.209904
1,6,0,0,1583,17452,0,61,8,0,0,...,63.0,1495.06,0.0,0.0,0.0,0.0,444991.0,21400.0,8.0,44.209904
2,11,1,0,23,115,1,18,11,0,0,...,20.0,2096.21,0.0,0.0,0.0,0.0,105737.0,16300.0,11.0,26.000000
3,11,0,0,1589,17451,0,24,21,0,0,...,26.0,1756.31,0.0,0.0,0.0,0.0,287022.0,72400.0,21.0,44.209904
4,6,0,0,1599,17450,0,34,16,0,0,...,36.0,172.21,0.0,0.0,0.0,0.0,234278.0,26700.0,16.0,44.209904


## 3.Split data into test and train datasets

In [56]:
from sklearn.model_selection import train_test_split

X = encoded
y = train_df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## 4.Create your first ML Model

In [ ]:
from sklearn.linear_model import LinearRegression

lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)

In [ ]:
lr_clf.predict([X_test.iloc[1,:]])

In [ ]:
lr_clf.predict(X_test)

In [ ]:
lr_clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000, max_depth=2, random_state=0)
rf_clf.fit(X_train, y_train) 

In [ ]:
rf_clf.score(X_test, y_test)

## 5.Creating a ML Pipeline

In [57]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
pipeline = make_pipeline(StandardScaler(),RandomForestClassifier(n_estimators=1000, max_depth=2, random_state=0) )

In [58]:
model = pipeline.fit(X_train, y_train)

In [59]:
predicted = model.predict(X_test)

In [60]:
print(predicted)

[0 0 0 ..., 0 0 0]


In [61]:
from sklearn import svm, metrics
metrics.classification_report(y_test, predicted)

'             precision    recall  f1-score   support\n\n          0       0.76      1.00      0.87     12229\n          1       0.00      0.00      0.00      3771\n\navg / total       0.58      0.76      0.66     16000\n'

## 6.Publishing the model to Watson ML

In [62]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [63]:
#get wml credentials from your service credentials tab under your ML Service
wml_credentials={
  
  "username": "*",
  "password": "*",
  "instance_id": "*",
  "url": "*"
}

In [64]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
import json

instance_details = client.service_instance.get_details()
print(json.dumps(instance_details, indent=2))

In [66]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Krishna", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "Krbalaga@in.ibm.com",
               client.repository.ModelMetaNames.NAME: "Loan eligibility model created on notebook"}

In [67]:
published_model = client.repository.store_model(model=model, meta_props=model_props, \
                                                training_data=X_train, training_target=y_train)

In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

In [ ]:
models_details = client.repository.list_models()

In [70]:
loaded_model = client.repository.load(published_model_uid)
print(loaded_model)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_im...timators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])


In [71]:
test_predictions = loaded_model.predict(X_test[:10])

In [72]:
print(test_predictions)

[0 0 0 0 0 0 0 0 0 0]


## 7.Deploying as a webservice

In [ ]:
created_deployment = client.deployments.create(published_model_uid, "Deployment of locally created scikit model")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

In [ ]:


deployments = client.deployments.get_details()

print(json.dumps(deployments, indent=2))



In [ ]:
deployment_url = client.deployments.get_url(created_deployment)

print(deployment_url)

In [81]:
scoring_payload = {"values": [list(X_test.iloc[1,:])]}

In [82]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [83]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "prediction",
    "probability"
  ],
  "values": [
    [
      0,
      [
        0.8074786744179258,
        0.19252132558207435
      ]
    ]
  ]
}
